In [3]:
import merge_utils as mu
import write_utils as wu
import parse_utils as pu
from operator import itemgetter

In [47]:
def recommendation_list(string):
    lista=[]
    split = string.split(',')
    for i in split:
        if i != '':
            lista.append((int(i)))
            
    return lista

def parse_submission(sc, path):
    rdd = sc.textFile(path)
    
    header = rdd.first()

    rdd = (rdd.filter(lambda x: x != header)
    .map(lambda x: x.split("\t"))
    .filter(lambda x: x[0] != '')
    .map(lambda x:(int(x[0]),recommendation_list(x[1])))
    )

    return rdd

def recommendation_list_with_value(string, value):
    updating_value = value
    lista=[]
    split = string.split(' ')
    for i in split:
        if i != '':
            updating_value -= 0.0001
            lista.append((int(i),updating_value))
            
    return lista

def psv(sc, path, value):
    rdd = sc.textFile(path)
    
    header = rdd.first()

    rdd = (rdd.filter(lambda x: x != header)
    .map(lambda x: x.split(","))
    .filter(lambda x: x[0] != '')
    .map(lambda x:(int(x[0]),recommendation_list_with_value(x[1], value)))
    )

    return rdd

def rdd_no_header(rdd):
    header = rdd.first()
    return rdd.filter(lambda x: x!=header)

def get_sim_dict(sim_rdd):
    return (rdd_no_header(sim_rdd)
                .map(lambda x:x.split("\t"))
                .filter(lambda x:x[0]!='' or x[1]!='')
                .map(lambda x:(int(x[0]),from_string_to_list(x[1], 30)))
               )

def get_weight_value(jobs, value):
    if len(jobs) > 0:
        return [(i[0],i[1]) for i in jobs[:50]]
    else:
        return []
    
def psvs(sc, path, value):
    rdd = sc.textFile(path)
    rec_rdd = (get_sim_dict(rdd)
               .map(lambda x: (x[0], get_weight_value(x[1], value)))
               .filter(lambda x: len(x[1]) > 0))
    
    return rec_rdd

def from_string_to_list(stringa,kNN=30):
    lista_stringa = stringa.replace("[","").replace("]","").replace("(","").split("),")
    lista_stringa = [i.replace(")","").split(", ") for i in lista_stringa]
    final=[]
    for tup in lista_stringa:
        if tup[0]!='':# or tup[1]!='':
            final.append((int(tup[0]),float(tup[1])))
    
    return sorted(final,key=itemgetter(1),reverse=True)[:kNN]

def filter_data(x):
    user = x[0]
    jobs = x[1]
    
    to_be_filter = union_dict.get(user)
    if to_be_filter != None:
        return [j for j in jobs if int(j[0]) not in to_be_filter]
    else:
        return jobs

In [36]:
interactions_rdd = parse_submission(sc, "../../Dataset/interactions.csv")
print interactions_rdd.take(1)

impressions_rdd = parse_submission(sc, "/Users/tommy/Downloads/impressions.csv")
print impressions_rdd.take(1)

union_rdd = impressions_rdd.union(interactions_rdd).reduceByKey(lambda x,y: x+y)
print union_rdd.take(1)

union_dict = dict(union_rdd.collect())

[(884155, [2004235])]
[(621311, [2015])]
[(1662980, [2015, 2015, 2572617, 1527982, 315676, 2510121, 777485, 1513997, 2572617, 475783])]


In [48]:
submission_name = "../../Submissions/tf_ervin.csv"
submission_rdd = psv(sc, submission_name, 1)

In [60]:
recommendation_rdd = submission_rdd.map(lambda x: (x[0], filter_data(x)))

In [61]:
wu.prepareSubmission(recommendation_rdd, "../../Submissions/FILTERED_tf_ervin")

Submission file ../../Submissions/FILTERED_tf_ervin created successfully
